# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [34]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [35]:
view_sentence_range = (0, 20)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 20:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.


Moe_Szyslak: Ah, isn't that nice. Now, there is a politician 

In [36]:
view_sentence_range

(0, 20)

In [37]:
scenes[0]

"Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.\nBart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.\nMoe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?\nMoe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.\nMoe_Szyslak: What's the matter Homer? You're not your normal effervescent self.\nHomer_Simpson: I got my problems, Moe. Give me another one.\nMoe_Szyslak: Homer, hey, you should not drink to forget your problems.\nBarney_Gumble: Yeah, you should only drink to enhance your social skills."

In [38]:
sentences_for_first_scene = scenes[0].split('\n')

In [39]:
sentences_for_first_scene

["Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.",
 'Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.',
 "Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?",
 "Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.",
 "Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.",
 'Homer_Simpson: I got my problems, Moe. Give me another one.',
 'Moe_Szyslak: Homer, hey, you should not drink to forget your problems.',
 'Barney_Gumble: Yeah, you should only drink to enhance your social skills.']

## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [153]:
from collections import Counter
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    
    count_of_words = Counter(text)
    vocab = sorted(count_of_words, key=count_of_words.get, reverse=True) # sort words
    vocab_to_int = {word: ii for ii, word in enumerate(vocab, 0)} # assign numbers to words based on its frequency
    
    int_to_vocab = dict()
    
    for index, word in enumerate(vocab_to_int, 0):
        int_to_vocab[vocab_to_int[word]] = word
    
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [154]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    tokens = {
        '.' : '||period||',
        ',' : '||comma||',
        '"' : '||quotation_mark||',
        ';' : '||semicolon||',
        '!' : '||exclamation_mark||',
        '?' : '||question_mark||',
        '(' : '||left_parentheses||',
        ')' : '||right_parentheses||',
        '--' : '||dash||',
        '\n' : '||return||'
    }
    
    return tokens

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [155]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [156]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

In [157]:
vocab_to_int

{'gave': 442,
 'cutting': 961,
 '50-60': 2896,
 'uh-huh': 664,
 'chauffeur': 2897,
 'officer': 2898,
 'government': 962,
 'powered': 2899,
 'medicine': 1900,
 'interested': 1901,
 'makes': 398,
 'folk': 1902,
 'fl': 2900,
 'wheels': 2901,
 'steam': 2902,
 'morose': 1903,
 'homunculus': 2903,
 'ladder': 2904,
 'refund': 1904,
 'aziz': 2905,
 'aggie': 2906,
 "haven't": 513,
 'count': 2907,
 'pardon': 1440,
 "soakin's": 2908,
 'thirty-five': 2909,
 'flag': 1905,
 'taunting': 2910,
 'hold': 379,
 'dizzy': 5473,
 'charged': 2912,
 'business': 349,
 'hootie': 2913,
 'investigating': 2914,
 "children's": 1931,
 'brown': 2916,
 'linda': 1906,
 'remains': 1907,
 "that'll": 847,
 'maximum': 2918,
 'puke-holes': 2919,
 'duel': 1908,
 'boozebag': 2920,
 'plenty': 1909,
 'wrapped': 2922,
 'fox': 1949,
 'experiments': 2925,
 'soup': 1953,
 'loathe': 2926,
 'windshield': 2927,
 "'cept": 2928,
 'writers': 1910,
 'police': 608,
 'guzzles': 2929,
 'booth': 2930,
 'lottery': 1589,
 'using': 778,
 'dinks'

In [158]:
int_to_vocab

{0: '||period||',
 1: '||return||',
 2: '||comma||',
 3: '||left_parentheses||',
 4: '||right_parentheses||',
 5: 'the',
 6: 'i',
 7: 'you',
 8: '||exclamation_mark||',
 9: 'moe_szyslak:',
 10: '||question_mark||',
 11: 'a',
 12: 'homer_simpson:',
 13: 'to',
 14: 'and',
 15: 'of',
 16: 'my',
 17: 'it',
 18: 'that',
 19: 'in',
 20: '||quotation_mark||',
 21: 'me',
 22: 'is',
 23: 'this',
 24: "i'm",
 25: 'for',
 26: 'your',
 27: 'homer',
 28: 'hey',
 29: 'on',
 30: 'moe',
 31: 'oh',
 32: 'no',
 33: 'lenny_leonard:',
 34: 'what',
 35: 'with',
 36: 'yeah',
 37: 'all',
 38: 'just',
 39: 'like',
 40: 'but',
 41: 'barney_gumble:',
 42: 'so',
 43: 'be',
 44: 'here',
 45: 'carl_carlson:',
 46: "don't",
 47: 'have',
 48: 'up',
 49: "it's",
 50: 'well',
 51: 'out',
 52: 'do',
 53: 'was',
 54: 'got',
 55: 'get',
 56: 'are',
 57: 'we',
 58: "that's",
 59: 'uh',
 60: 'one',
 61: "you're",
 62: 'now',
 63: 'not',
 64: 'can',
 65: 'know',
 66: '||dash||',
 67: 'at',
 68: 'right',
 69: '/',
 70: 'how'

In [159]:
int_to_vocab[10]

'||question_mark||'

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [160]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [161]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    input = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    
    return input, targets, learning_rate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [162]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    cell = tf.contrib.rnn.MultiRNNCell([lstm]) # I assume that RNN is going to be one-layer
    # If I see the need of introducing more layers, I need to modify this function and
    # add an additinal hyper-parameter
    
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, tf.identity(initial_state, name='initial_state')


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [163]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # initializing the embedding layer with uniform distribution
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    
    embed_layer = tf.nn.embedding_lookup(embedding, input_data)
    
    return embed_layer


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [164]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # inputs variable is confusing here. Inputs actually is the embedding. 
    # Question: I am still confused how we are going to create this cell without the initial_state variable
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    
    return outputs, tf.identity(final_state, name='final_state')


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [165]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    embed_layer = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell, embed_layer)
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None)
    
    return logits, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [166]:
test_batch = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [167]:
batch_size = 3
sequence_length = 2
batch = batch_size * sequence_length 
no_of_batches = len(test_batch) // (batch_size * sequence_length)

In [168]:
def get_batches2(int_text, batch_size, seq_length):
    no_of_batches = len(int_text) // (batch_size * seq_length)

    x = np.array(int_text[: no_of_batches * batch_size * seq_length])
    print(x)
    y = np.array(int_text[1 : no_of_batches * batch_size * seq_length + 1])
    y[-1] = x[0]
    print(y)
    
    x_reshaped = x.reshape(batch_size, -1)
    print("reshaped_x", x_reshaped)
    y_reshaped = y.reshape(batch_size, -1)
    print("reshaped_y", y_reshaped)
    
    batch_x = np.split(x_reshaped, no_of_batches, 1)
    print(batch_x)
    batch_y = np.split(y_reshaped, no_of_batches, 1)
    print(batch_y)
    return np.array(list(zip(batch_x, batch_y)))

In [169]:
x = get_batches2(test_batch, 3, 2)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18  1]
reshaped_x [[ 1  2  3  4  5  6]
 [ 7  8  9 10 11 12]
 [13 14 15 16 17 18]]
reshaped_y [[ 2  3  4  5  6  7]
 [ 8  9 10 11 12 13]
 [14 15 16 17 18  1]]
[array([[ 1,  2],
       [ 7,  8],
       [13, 14]]), array([[ 3,  4],
       [ 9, 10],
       [15, 16]]), array([[ 5,  6],
       [11, 12],
       [17, 18]])]
[array([[ 2,  3],
       [ 8,  9],
       [14, 15]]), array([[ 4,  5],
       [10, 11],
       [16, 17]]), array([[ 6,  7],
       [12, 13],
       [18,  1]])]


In [170]:
x

array([[[[ 1,  2],
         [ 7,  8],
         [13, 14]],

        [[ 2,  3],
         [ 8,  9],
         [14, 15]]],


       [[[ 3,  4],
         [ 9, 10],
         [15, 16]],

        [[ 4,  5],
         [10, 11],
         [16, 17]]],


       [[[ 5,  6],
         [11, 12],
         [17, 18]],

        [[ 6,  7],
         [12, 13],
         [18,  1]]]])

In [171]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    no_of_batches = len(int_text) // (batch_size * seq_length)

    x = np.array(int_text[: no_of_batches * batch_size * seq_length])
    y = np.array(int_text[1 : no_of_batches * batch_size * seq_length + 1])
    y[-1] = x[0]

    x_reshaped = x.reshape(batch_size, -1)
    y_reshaped = y.reshape(batch_size, -1)
    
    batch_x = np.split(x_reshaped, no_of_batches, 1)
    batch_y = np.split(y_reshaped, no_of_batches, 1)

    return np.array(list(zip(batch_x, batch_y)))


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [188]:
# Number of Epochs
num_epochs = 200
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 200
# Embedding Dimension Size
embed_dim = 100
# Sequence Length
seq_length = 25
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 20

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [189]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [190]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/10   train_loss = 8.822
Epoch   2 Batch    0/10   train_loss = 5.838
Epoch   4 Batch    0/10   train_loss = 5.106
Epoch   6 Batch    0/10   train_loss = 4.573
Epoch   8 Batch    0/10   train_loss = 4.155
Epoch  10 Batch    0/10   train_loss = 3.805
Epoch  12 Batch    0/10   train_loss = 3.517
Epoch  14 Batch    0/10   train_loss = 3.268
Epoch  16 Batch    0/10   train_loss = 3.048
Epoch  18 Batch    0/10   train_loss = 2.905
Epoch  20 Batch    0/10   train_loss = 2.800
Epoch  22 Batch    0/10   train_loss = 2.600
Epoch  24 Batch    0/10   train_loss = 2.358
Epoch  26 Batch    0/10   train_loss = 2.231
Epoch  28 Batch    0/10   train_loss = 2.179
Epoch  30 Batch    0/10   train_loss = 2.035
Epoch  32 Batch    0/10   train_loss = 1.939
Epoch  34 Batch    0/10   train_loss = 1.819
Epoch  36 Batch    0/10   train_loss = 1.733
Epoch  38 Batch    0/10   train_loss = 1.757
Epoch  40 Batch    0/10   train_loss = 1.622
Epoch  42 Batch    0/10   train_loss = 1.531
Epoch  44 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [191]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [192]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [193]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    InputTensor = loaded_graph.get_tensor_by_name('input:0')
    InitialStateTensor = loaded_graph.get_tensor_by_name('initial_state:0')
    FinalStateTensor = loaded_graph.get_tensor_by_name('final_state:0')
    ProbsTensor = loaded_graph.get_tensor_by_name('probs:0')
    
    return InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [194]:
prob = np.array([0.1,0.2, 0.7, 0.7])

In [195]:
np.max(prob)

0.69999999999999996

In [196]:
itemindex = np.where(prob == np.max(prob))

In [197]:
itemindex[0][0]

2

In [198]:
int_to_vocab[1]

'||return||'

In [199]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    max_prob = np.max(probabilities)
    item_index = np.where(probabilities == max_prob)
    first_appearance = item_index[0][0]
    
    return int_to_vocab[first_appearance]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [200]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak:(sighs) i didn't want to have to do this...
homer_simpson:(chuckles at injury)" that's why they don't talk much.
moe_szyslak: hey, hey, hey, hey! that plank's only for comin' in! you've got a choice: you can use the restroom.
apu_nahasapeemapetilon:(sighs) my one day off and it's miserable.
manjula_nahasapeemapetilon: this is my day...
homer_simpson:(eyes wide, gasps) you and me you need to sing for me.
seymour_skinner:(shaking hands) principal seymour skinner.
apu_nahasapeemapetilon: apu nahasapeemapetilon.
nigel_bakerbutcher: how do you think i could never lie to you, and the tv's japanese(points to tv)...
crowd:(chanting) barney forever, wiggum never! barney forever, homer, you wanna go bowling next sunday?
homer_simpson: i'unno. what am i, superdad?
moe_szyslak: yeah, a kid should be real simple to kill.
moe_szyslak:(sobs like teenage thought i'd not be too bad with lenny more beer at the


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.